##### Import

In [2]:
import rx
from rx import operators as ops
from urllib3 import Retry
from datetime import datetime
from influxdb_client import InfluxDBClient, Point, WritePrecision, WriteOptions
from influxdb_client.client.write_api import SYNCHRONOUS
from collections import OrderedDict
from csv import DictReader
from os import getcwd
from os.path import join, exists, isfile, isdir, abspath, pardir
from time import time

##### Configs

In [3]:
# You can generate an API token from the "API Tokens Tab" in the UI
# token = "RtWgHEkMiipthr5IkT1RpRed4ZXRH-9Q8YuEzZ3pdkjgelVA83D1SZ5tF9V_NY3u7W5-lAl-GuVUC3n3mS1cCQ=="
token = "RtWgHEkMiipthr5IkT1RpRed4ZXRH-9Q8YuEzZ3pdkjgelVA83D1SZ5tF9V_NY3u7W5-lAl-GuVUC3n3mS1cCQ=="
org = "ulb"
bucket = "advanced_db"
port=8086
url="http://localhost:{}".format(port)

In [4]:
import pandas as pd

In [5]:
file_name = "trades_btc_1_day.csv"
file_path = join(abspath(join(getcwd(), pardir)), "data", "data_trades", file_name)

In [6]:
df = pd.read_csv(file_path)

In [35]:
def parse_trade_row(row: OrderedDict):
    return Point("trades").tag("symbol", "{}".format(row['symbol'])) \
        .field("side", 1 if row['side'] == 'buy' else -1) \
        .field("price", float(row['price'])) \
        .field("amount", float(row['amount'])) \
        .time(datetime.fromtimestamp(float(row['time_stamp'])))

In [36]:
data = list(map(parse_trade_row, DictReader(open(file_path, 'r'))))

In [37]:
for i in range(0, 10):
    print(data[i].to_line_protocol())

trades,symbol=BTC-USDT amount=0.00868,price=57587.04,side=1i 1637370000168152000
trades,symbol=BTC-USDT amount=0.29039,price=57587.03,side=-1i 1637370000418139000
trades,symbol=BTC-USDT amount=0.00018,price=57587.03,side=-1i 1637370000668128000
trades,symbol=BTC-USDT amount=0.00043,price=57584.15,side=-1i 1637370000918115000
trades,symbol=BTC-USDT amount=0.00617,price=57584.15,side=-1i 1637370001168103000
trades,symbol=BTC-USDT amount=0.00505,price=57584.16,side=1i 1637370001418091000
trades,symbol=BTC-USDT amount=0.07089,price=57587.04,side=1i 1637370001668079000
trades,symbol=BTC-USDT amount=0.36741,price=57588.76,side=1i 1637370001918067000
trades,symbol=BTC-USDT amount=0.04585,price=57587.03,side=-1i 1637370002168055000
trades,symbol=BTC-USDT amount=0.0111,price=57587.03,side=-1i 1637370002418043000


In [29]:
d = DictReader(open(file_path, 'r'))

In [30]:
list(map(parse_trade_row, d))

 ...]

In [26]:
# with InfluxDBClient(url=url, token=token, org=org) as client:
#     with client.write_api(write_options=WriteOptions(batch_size=50_000, flush_interval=10_000)) as write_api:
#         start_time = time()
#         write_api.write(bucket=bucket, record=data)
#         print("Took {} seconds to load records in {}".format( time() - start_time, file_name))

Took 0.00027489662170410156 seconds to load records in trades_btc_1_day.csv


In [19]:
_data_ = list()

for index, row in enumerate(DictReader(open(file_path, 'r'))):
    # point = parse_trade_row(row)
    print(datetime.fromtimestamp(float(row['time_stamp'])))
    # print("Rows: {}".format(point))
    if index == 1:
        break

2021-11-20 01:00:00.168152
2021-11-20 01:00:00.418139


In [12]:
client = InfluxDBClient(url=url, token=token, org=org)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

p = Point("city_temp").tag("location", "brussels").field("temperature", 25.3)

write_api.write(bucket="demo", record=data)

## using Table structure
tables = query_api.query('from(bucket:"demo") |> range(start: -10m)')

In [30]:
data

In [29]:
write_api.write(bucket="demo", record=data)

In [15]:
for table in tables:
    print(table)
    for row in table.records:
        print (row.values)


# ## using csv library
# csv_result = query_api.query_csv('from(bucket:"my-bucket") |> range(start: -10m)')
# val_count = 0
# for row in csv_result:
#     for cell in row:
#         val_count += 1

FluxTable() columns: 9, records: 1
{'result': '_result', 'table': 0, '_start': datetime.datetime(2021, 11, 21, 19, 22, 43, 356533, tzinfo=tzutc()), '_stop': datetime.datetime(2021, 11, 21, 19, 32, 43, 356533, tzinfo=tzutc()), '_time': datetime.datetime(2021, 11, 21, 19, 32, 42, 783335, tzinfo=tzutc()), '_value': 25.3, '_field': 'temperature', '_measurement': 'city_temp', 'location': 'brussels'}


In [16]:
csv_result = query_api.query_csv('from(bucket:"demo") |> range(start: -10m)')
val_count = 0
for row in csv_result:
    for cell in row:
        val_count += 1

In [28]:
tables[0].get_group_key()

[FluxColumn(2, label='_start', data_type='dateTime:RFC3339', group=True, default_value=''),
 FluxColumn(3, label='_stop', data_type='dateTime:RFC3339', group=True, default_value=''),
 FluxColumn(6, label='_field', data_type='string', group=True, default_value=''),
 FluxColumn(7, label='_measurement', data_type='string', group=True, default_value=''),
 FluxColumn(8, label='location', data_type='string', group=True, default_value='')]

In [23]:
query_api = client.query_api()

In [15]:
query = 'from(bucket:"advanced_db")' \
        ' |> range(start: 0, stop: now())' \
        ' |> filter(fn: (r) => r._measurement == "trades")'
result = client.query_api().query(query=query)

In [16]:
result

[]